# IMD3004 - IA Generativa

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

Código adaptado de:

[1] Sebastian Raschka,Yuxi (Hayden) Liu e e Vahid Mirjalili. Machine Learning with PyTorch and Scikit-Learn. [capítulo 17] [(Github link)](https://github.com/rasbt/machine-learning-book)

Referências adicionais:

[1] T. Kynkäänniemi et al., “Improved Precision and Recall Metric for Assessing Generative Models,” Adv. Neural Inf. Process. Syst., vol. 32, no. NeurIPS, Apr. 2019. [(Paper link)](https://proceedings.neurips.cc/paper_files/paper/2019/file/0234c510bc6d908b28c70ff313743079-Paper.pdf)





## Importações e Ambiente

In [ ]:
!pip install prdc torchinfo

In [ ]:
import torch                                # Importa a biblioteca principal do PyTorch, usada para computação com tensores
from torch import nn                        # Importa o módulo 'nn' do PyTorch, utilizado para criar e treinar redes neurais
from torch.utils.data import Subset, DataLoader  # Subset permite criar subconjuntos de datasets, e DataLoader facilita o carregamento de dados em mini-lotes.
from torchvision import datasets            # Importa o módulo de conjuntos de dados do torchvision, que fornece acesso a conjuntos de dados populares como MNIST, CIFAR, etc. para visão computacional
from torchvision import transforms          # Importa o módulo de transforms do torchvision, usado para pré-processar imagens (conversão em tensor, normalização, etc.).
from torchinfo import summary               # Importa a função summary da biblioteca torchinfo, usada para exibir a arquitetura do modelo
import numpy as np                          # Importa a biblioteca NumPy, usada para operações numéricas eficientes com arrays e matrizes.
import matplotlib.pyplot as plt             # Importa o módulo de visualização matplotlib para gerar gráficos.
from IPython.display import Image           # Importa a classe Image do IPython, usada para exibir imagens diretamente dentro de notebooks.
import os                                   # Importa o módulo padrão do Python para interagir com o sistema operacional (ex.: criar pastas, manipular caminhos de arquivos).
from prdc import compute_prdc               # Importa a função compute_prdc, usada para calcular as métricas de Precisão e Revocação para modelos generativos.
import seaborn as sns                       # Importa a biblioteca de visualização seaborn.
import pandas as pd                         # Importa a biblioteca pandas, usada para manipulação e análise de dados em estruturas como DataFrames.
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" # Define uma variável de ambiente para evitar conflitos de biblioteca durante a execução em alguns ambientes.

In [ ]:
# Para figuras geradas pelo matplotlib serem exibidas diretamente no notebook
%matplotlib inline

In [ ]:
print("PyTorch version:", torch.__version__)            # Mostra a versão instalada do PyTorch
print("GPU Available:", torch.cuda.is_available())      # Indica se há GPU CUDA disponível
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu" # Usa acelerador disponível (CUDA/MPS/XPU) ou então usa CPU

In [ ]:
if not os.path.exists('files'):
    os.makedirs('files')

## Dados

In [ ]:
image_path = './'                                         # Define o diretório raiz onde o conjunto de dados MNIST será armazenado.

transform = transforms.Compose([                           # Define uma sequência de transformações a serem aplicadas às imagens do dataset.
    transforms.ToTensor(),                                 # Converte as imagens PIL em tensores PyTorch.
    transforms.Normalize(mean=(0.5), std=(0.5)),           # Normaliza os valores dos pixels para a faixa [-1, 1], com média 0.5 e desvio 0.5.
])

mnist_dataset = datasets.MNIST(root=image_path,            # Cria o objeto do dataset MNIST.
                               train=True,                 # Define que serão carregadas as imagens do conjunto de treinamento.
                               transform=transform,        # Aplica as transformações definidas acima a cada imagem carregada.
                               download=True)              # Faz o download do dataset caso ele não esteja presente no diretório especificado.

mnist_test = datasets.MNIST(root=image_path,               # Cria o objeto do dataset MNIST.
                               train=False,                # Define que serão carregadas as imagens do conjunto de teste.
                               transform=transform,        # Aplica as transformações definidas acima a cada imagem carregada.
                               download=True)              # Faz o download do dataset caso ele não esteja presente no diretório especificado.

print(f'mnist_dataset | mnist_test: {type(mnist_dataset)} | {type(mnist_test)}.')

example, label = next(iter(mnist_dataset))                 # Obtém o primeiro par (imagem, rótulo) do dataset MNIST.
print(f'Min: {example.min()} Max: {example.max()}')        # Exibe os valores mínimo e máximo do tensor da imagem (após normalização).
print(example.shape)                                       # Exibe o formato do tensor da imagem (para MNIST: [1, 28, 28]).

## Classificador

### Modelo

In [ ]:
# Classificador dividido em dois nn.Sequential: feature_extractor + classifier_head

# Extrator de features
feature_extractor = nn.Sequential(
    nn.Conv2d(1, 32, 3), nn.ReLU(), nn.MaxPool2d(2),    # -> (B,32,13,13)
    nn.Conv2d(32, 64, 3), nn.ReLU(), nn.MaxPool2d(2),   # -> (B,64,5,5)
    nn.Conv2d(64, 128, 3), nn.ReLU(),                   # -> (B,128,3,3)
    nn.AdaptiveAvgPool2d(1),                            # -> (B,128,1,1)
    nn.Flatten()                                        # -> (B,128)
).to(device)

# Cabeça classificadora
classifier_head = nn.Sequential(
    nn.Linear(128, 10)                                  # -> (B,10)
).to(device)

# Modelo completo
full_model = nn.Sequential(
    feature_extractor,
    classifier_head
).to(device)

In [ ]:
# Extrator de features
summary(feature_extractor, input_size=(1, 1, 28, 28))

In [ ]:
# Cabeça classificadora
summary(classifier_head, input_size=(1, 128))

In [ ]:
# Modelo completo
summary(full_model, input_size=(1, 1, 28, 28))

### Treinamento do classificador

In [ ]:
## Função de perda e otimizadores
loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(full_model.parameters(), lr=1e-3)

## Preparação do conjunto de dados
train_dl = DataLoader(mnist_dataset, batch_size=128, shuffle=True,  drop_last=True)
test_dl  = DataLoader(mnist_test,    batch_size=512, shuffle=False, drop_last=False)

In [ ]:
def train_one_epoch(dataloader, model, loss_fn, optimizer):
    model.train()                                               # Coloca o modelo em modo de treinamento (ativa dropout, batchnorm, etc.)
    total_loss, total, correct = 0.0, 0, 0                      # Inicializa acumuladores de perda, número de exemplos e acertos
    for x, y in dataloader:                                     # Itera pelos lotes de dados
        x, y = x.to(device), y.to(device)                       # Move entradas e rótulos para o dispositivo (CPU/GPU)
        logits = model(x)                                       # Calcula as predições do modelo
        loss = loss_fn(logits, y)                               # Calcula a perda entre predições e rótulos
        optimizer.zero_grad()                                   # Zera gradientes acumulados
        loss.backward()                                         # Calcula gradientes via backpropagation
        optimizer.step()                                        # Atualiza os parâmetros do modelo
        total_loss += loss.item()                               # Acumula a perda do lote
        correct    += (logits.argmax(1) == y).sum().item()      # Conta acertos comparando previsão com rótulo
        total      += x.size(0)                                 # Conta número de exemplos processados
    return total_loss/total, correct/total                      # Retorna perda média por exemplo e acurácia

In [ ]:
@torch.no_grad()                                                 # Desativa o cálculo de gradientes (avaliação mais rápida e com menor uso de memória)
def evaluate(dataloader, model, loss_fn):
    model.eval()                                                 # Coloca o modelo em modo de avaliação (desativa dropout, batchnorm usa estatísticas fixas)
    total_loss, total, correct = 0.0, 0, 0                       # Inicializa acumuladores de perda, número de exemplos e acertos
    for x, y in dataloader:                                      # Itera pelos lotes de dados
        x, y = x.to(device), y.to(device)                        # Move entradas e rótulos para o dispositivo (CPU/GPU)
        logits = model(x)                                        # Calcula as predições do modelo
        loss = loss_fn(logits, y)                                # Calcula a perda entre predições e rótulos
        total_loss += loss.item()                                # Acumula a perda do lote
        correct    += (logits.argmax(1) == y).sum().item()       # Conta acertos comparando previsão com rótulo
        total      += x.size(0)                                  # Conta número de exemplos processados
    return total_loss/total, correct/total                       # Retorna perda média por exemplo e acurácia

In [ ]:
n_epocas = 10                                                                                       # Define o número total de épocas de treinamento
for ep in range(n_epocas):                                                                          # Loop principal sobre as épocas
    tr_loss, tr_acc = train_one_epoch(train_dl, full_model, loss_fn, optimizer)                     # Executa uma época de treinamento e retorna perda/acurácia
    te_loss, te_acc = evaluate(test_dl, full_model, loss_fn)                                        # Avalia o modelo no conjunto de teste e retorna perda/acurácia
    print(f"Época {ep + 1}/{n_epocas} | Treinamento: perda={tr_loss:.4f} acc={tr_acc:.3f} | "
          f"Teste: perda={te_loss:.4f} acc={te_acc:.3f}")                                          # Imprime resultados formatados de treino e teste


## Funções Auxiliares

In [ ]:
def create_noise(batch_size, z_size, mode_z):
    """Gera um tensor de ruído (vetor latente)."""
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size, device=device) * 2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size, device=device)
    return input_z

In [ ]:
@torch.no_grad()
def get_features(dataset, feature_extractor, N: int, batch_size: int = 512) -> np.ndarray:
    """Extrai features de um conjunto de dados real."""
    feature_extractor.eval()
    subset = Subset(dataset, range(min(N, len(dataset))))
    loader = DataLoader(subset, batch_size=batch_size, shuffle=False, drop_last=False)
    feats_list = []
    for x, _ in loader:
        x = x.to(device)
        emb = feature_extractor(x)
        feats_list.append(emb.cpu().numpy().astype(np.float32))
    return np.concatenate(feats_list, axis=0)

In [ ]:
@torch.no_grad()
def get_features_from_generator(gen_model, feature_extractor, N: int, z_dim: int, mode_z: str, batch_size: int, image_size: tuple) -> np.ndarray:
    """Gera imagens e extrai suas features."""
    gen_model.eval()
    feature_extractor.eval()
    feats = []
    remaining = N
    while remaining > 0:
        cur = min(batch_size, remaining)
        z = create_noise(cur, z_dim, mode_z)
        # Check if the model is a conditional WGAN and requires labels
        if hasattr(gen_model, 'label_embedding'):
            labels = torch.randint(0, 10, (cur,), device=device) # Generate random labels for MNIST (0-9)
            imgs = gen_model(z, labels).view(cur, 1, *image_size)
        else:
            # Garante que a saída do gerador tenha o formato [B, C, H, W]
            imgs = gen_model(z).view(cur, 1, *image_size)

        emb = feature_extractor(imgs)
        feats.append(emb.cpu().numpy().astype(np.float32))
        remaining -= cur
    return np.concatenate(feats, axis=0)

In [ ]:
def mpr_curve_by_k(real_feats, fake_feats, ks):
    """Calcula precisão e revocação para uma lista de valores k."""
    prec, rec = [], []
    for k in ks:
        print(f'Calculando MPR para k={k}...')
        m = compute_prdc(real_feats, fake_feats, nearest_k=k)
        prec.append(m['precision'])
        rec.append(m['recall'])
    return np.array(rec), np.array(prec)

In [ ]:
def mpr_sweep_Nk(
    mnist_test_dataset,
    gen_model,
    feature_extractor,
    Ns=(1000, 2500, 5000, 7500, 10000),
    ks=(3, 5, 10),
    *,
    model_name='GAN',
    batch_size=512,
    z_dim=100,
    mode_z='uniform',
    image_size=(28, 28)
) -> pd.DataFrame:
    """
    Executa uma varredura (sweep) sobre diferentes valores de N (nº de amostras)
    e k (vizinhos mais próximos) para calcular a precisão e a revocação.

    Retorna:
        pd.DataFrame: Um DataFrame com os resultados para cada combinação de N e k.
    """
    linhas = []
    for N in Ns:
        # Extrai as features uma vez para cada valor de N
        print(f"  [N={N}] Extraindo features reais e geradas...")
        real_feats = get_features(mnist_test_dataset, feature_extractor, N=N, batch_size=batch_size)
        fake_feats = get_features_from_generator(
            gen_model, feature_extractor, N, z_dim, mode_z, batch_size, image_size
        )

        # Calcula as métricas para diferentes valores de k
        for k in ks:
            m = compute_prdc(
                real_features=real_feats.astype('float32'),
                fake_features=fake_feats.astype('float32'),
                nearest_k=k
            )
            # Adiciona os resultados à lista
            linhas.append({
                'modelo':    model_name,
                'N':         int(N),
                'k':         int(k),
                'precision': float(m['precision']),
                'recall':    float(m['recall'])
            })
            print(f"    [modelo={model_name}] N={N} | k={k} -> Precisão={m['precision']:.4f} | Revocação={m['recall']:.4f}")

    return pd.DataFrame(linhas)

In [ ]:
def plot_mpr_df(df: pd.DataFrame):
    """
    Plota os resultados da varredura de Precisão e Revocação a partir de um DataFrame.

    Cria uma linha de gráficos (Precisão vs N, Revocação vs N) para cada modelo
    encontrado no DataFrame.
    """
    modelos = df['modelo'].unique().tolist()
    n_models = len(modelos)

    fig, axes = plt.subplots(
        n_models, 2,
        figsize=(14, 5 * n_models),
        dpi=100,
        constrained_layout=True,
        squeeze=False # Garante que 'axes' seja sempre 2D
    )

    for i, modelo in enumerate(modelos):
        df_model = df[df['modelo'] == modelo].copy()

        # --- Gráfico da Esquerda: Precisão ---
        ax_left = axes[i, 0]
        sns.lineplot(
            data=df_model, x='N', y='precision',
            hue='k', marker='o', ax=ax_left, palette='viridis'
        )
        ax_left.set_title(f'Precisão vs N ({modelo})', fontsize=14)
        ax_left.set_xlabel('N (número de imagens)', fontsize=12)
        ax_left.set_ylabel('Precisão (Qualidade)', fontsize=12)
        ax_left.set_ylim(0, 1) # Fixa o eixo Y entre 0 e 1
        ax_left.grid(True, linestyle='--', alpha=0.5)
        ax_left.legend(title='k')

        # --- Gráfico da Direita: Revocação ---
        ax_right = axes[i, 1]
        sns.lineplot(
            data=df_model, x='N', y='recall',
            hue='k', marker='o', ax=ax_right, palette='viridis'
        )
        ax_right.set_title(f'Revocação vs N ({modelo})', fontsize=14)
        ax_right.set_xlabel('N (número de imagens)', fontsize=12)
        ax_right.set_ylabel('Revocação (Diversidade)', fontsize=12)
        ax_right.set_ylim(0, 1) # Fixa o eixo Y entre 0 e 1
        ax_right.grid(True, linestyle='--', alpha=0.5)
        ax_right.legend(title='k')

    plt.show()

In [ ]:
def load_model(model_path, device):
    """Carrega um modelo salvo e o configura para o modo de avaliação."""
    loaded_model = torch.jit.load(model_path, map_location=device)
    loaded_model.eval()
    return loaded_model

## Tarefa

1. Treinar os seguintes modelos utilizando o conjunto de **treinamento** MNIST:

- GAN (com camadas completamente conectadas, isto é, GAN **NÃO** convolucional)
- DCGAN (convolucional)
- WGAN (convolucional)

Observação: A utilização dos laboratórios anteriores é permitida. Nesse caso, salvar os modelos treinados e carregar tais modelos aqui.

2. Comparar os modelos utilizando as curvas de MPR para diferentes valores do parâmetro `k` (mostre os gráficos com legendas para os modelos). Utilize o conjunto de **teste** MNIST para a aproximação da variedade dos dados reais.

- Qual o comportamento da precisão e revocação ao se aumentar o valor de k?

3. Varie o número de imagens `N=[1000, 2500, 5000, 7500, 10000]` utilizadas (utilize o mesmo valor para o número de imagens reais e o número de imagens geradas) no cálculo da precisão e revocação. Observe o comportamento das curvas de precisão e revocação (para `k=[3, 5, 10]`) para cada modelo.

- Mostrar 3 gráficos, um para cada modelo.

**Entregáveis**:
1. Notebook `.ipynb`.
2. Relatório `.pdf`:

    - Reporte e comente os resultados no relatório.

    - Incluir gráficos gerados.


### 1:

In [ ]:
generator_gan = load_model('files/mnist_generator_nonconv_gan.pt', device)
generator_dcgan = load_model('files/mnist_generator_conv_dcgan.pt', device)
generator_wgan = load_model('files/mnist_conditional_conv_wgan.pt', device)

### 2:

In [ ]:
image_size = (28, 28)
z_size = 100
mode_z = 'uniform'
batch_size_eval = 512

N = 10_000
ks = list(range(1, 11))
feature_extractor = full_model[0]
dataset = mnist_test

generators = {
    'GAN (FC)': generator_gan,
    'DCGAN': generator_dcgan,
    'WGAN': generator_wgan
}

results = {}

In [ ]:
real_features = get_features(dataset, feature_extractor, N, batch_size_eval)
print(f"Shape das features reais: {real_features.shape}")

In [ ]:
for name, model in generators.items():
    print(f"\nProcessando o modelo: {name}")

    # Extrair features das imagens geradas pelo modelo atual
    fake_features = get_features_from_generator(
        model,
        feature_extractor,
        N,
        z_size,
        mode_z,
        batch_size_eval,
        image_size
    )

    # Calcular a curva MPR variando k
    rec, prec = mpr_curve_by_k(real_features, fake_features, ks)
    results[name] = {'recall': rec, 'precision': prec}

In [ ]:
plt.figure(figsize=(10, 8))
for name, data in results.items():
    plt.plot(data['recall'], data['precision'], marker='o', label=name)

# Anotar pontos de k para o WGAN (ou o último modelo da lista)
if 'WGAN' in results:
    wgan_results = results['WGAN']
    for r, p, k in zip(wgan_results['recall'], wgan_results['precision'], ks):
        if k in [1, 3, 5, 7, 10]:
            plt.text(r + 0.005, p, f'k={k}')

plt.xlim(0, 1); plt.ylim(0, 1)
plt.xlabel('Revocação ( mede a diversidade )', fontsize=14)
plt.ylabel('Precisão ( mede a qualidade )', fontsize=14)
plt.title('Curva MPR: Comparação de Modelos (N=10.000)', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(fontsize=12)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

### 3:

In [ ]:
Ns_sweep = (1000, 2500, 5000, 7500, 10000)
ks_sweep = (3, 5, 10)

all_dfs = []

for name, model in generators.items():
    print(f"--- Iniciando varredura para o modelo: {name} ---")
    df_model = mpr_sweep_Nk(
        mnist_test_dataset=mnist_test,
        gen_model=model,
        feature_extractor=feature_extractor,
        Ns=Ns_sweep,
        ks=ks_sweep,
        model_name=name,
        batch_size=batch_size_eval,
        z_dim=z_size,
        mode_z=mode_z
    )
    all_dfs.append(df_model)
    print(f"--- Varredura para {name} concluída ---\n")

df_final = pd.concat(all_dfs, ignore_index=True)
plot_mpr_df(df_final)